<h1> A Brief Introduction to the SCP Framework</h1>
    
<p>
    The mathematical considerations and properties of the SCP framework are discussed in detail in my Master's Thesis. This file is intended to introduce a step by step procedure for modelling the Suppression Task using the SCP Framework.
</p>

<p> Add a path to the SCP Framework
   </p>

In [1]:
import copy
import sys
sys.path.append("/SCPFramework") 

<p>
Next, we import the major modules used by the SCP Framework.
<ul>
    <li><b>basicLogic</b>: an implementation of basic 2 and 3 valued logic.</li>
  <li><b>epistemicState</b>: handles creation of epistemic states (lists of structural variables).</li>
  <li><b>SCP_Task</b>: creates the SCP Task, from which SCPs may be generated.</li>
    <li><b>scpNotationParser</b>: used to translate string into basicLogic clauses.</li>
    <li><b>CTM</b>: A sequence of congitive operations which begin with an initial epistemic state.</li>
    <li><b>CognitiveOperation</b>: pipes in the pipeline that is a CTM, transforms an input state point into some output state point.</li>
    <li><b>StatePointOperations</b>: Generally useful SCP operations, such as testing strict and lenient modelling.</li>
</ul> 
</p>

In [2]:
from SCPFramework import basicLogic
from SCPFramework import epistemicState
from SCPFramework import SCP_Task
from SCPFramework import scpNotationParser
from SCPFramework import CTM
from SCPFramework import CognitiveOperation
from SCPFramework import StatePointOperations

<h2> The Suppression Task</h2>
<p>We will be modelling the Suppression Task to demonstrate the usefulness of the SCP Framework. 
The formulation we will be using is as follows:
</p>
<ul>
<li>If she has an essay to write she will study late in the library.</li>
<li>If the library is open she will study late in the library.</li>
<li>She has an essay to write.</li>
</ul>

<h2> Creating The Inital State Point</h2>
<p> The initial epistemic state consists of a list of <i>epistemicState</i> objects, each one corresponding to the unique information we have about the world in that instant. </p>

<h3>  The Possible World V</h3>
<p> The possible world of an epistemic state p is given by p[V] and represents the truth values assigned to that variable in its interpretation up that this point. We begin by defining V as the set of variables e,l, and o. All initially set to unknown. </p>

In [3]:
#STARTING VARIABLES
# e: she has an essay to write
e = basicLogic.atom('e')
# l: she will study late in the library
l = basicLogic.atom('l')
# o: the library is open
o = basicLogic.atom('o')

<h3>  The Set of Conditionals and Set of rules </h3>
<p> In the first case of the suppression task we are aware only of the conditional (l|e). We are also aware of the unconditional fact that she has an essay to write. </p>

In [4]:
delta_el=["( l | e )"]
#interpret the string set of the conditional as basicLogic
delta_el_AsLogic = scpNotationParser.stringListToBasicLogic(delta_el)
S1 = ["( e <- T )"]
#interpret the string set of the propositional rules as basicLogic
S1AsLogic = scpNotationParser.stringListToBasicLogic(S1)

<h3> The first epistemic state </h3>
<p>The first epistemic state we will consider is the case where an individual has only the information in delta_el and S1, we call this the 'el' case. It contains the structural variables S, Delta, and V.</p>

In [5]:
basePoint_el=epistemicState.epistemicState('el')

S1AsLogic = scpNotationParser.stringListToBasicLogic(S1)
basePoint_el['S']=S1AsLogic
basePoint_el['Delta']=delta_el_AsLogic
basePoint_el['V']=[e,l]

<h3> The second epistemic state </h3>
<p>The second epistemic state we will consider is the case where an individual has the information in delta_el and S1 as well as the extra conditional that she will strudy late in the library if the library is open, we call this the 'elo' case. It contains the structural variables S, Delta, and V.</p>

In [6]:
#The elo case expressed as the addition of information to the el case
basePoint_elo=copy.deepcopy(basePoint_el)
basePoint_elo.setName('elo')
#The possible starting states for the SCP
extraConditional=["( l | o )"]
extraConditionalAsLogic = scpNotationParser.stringListToBasicLogic(extraConditional)
basePoint_elo['Delta']=basePoint_elo['Delta']+extraConditionalAsLogic
basePoint_elo['V']=basePoint_elo['V']+[o]

<h3> The initial state point </h3>
<p>The initial state of any SCP to model the suppression task will use both el and elo  and show that, by using some set of cognitive operations,
it is possible to derive an inference in el that is not derived in elo.</p>

In [7]:
s_i=[basePoint_el,basePoint_elo]

In [8]:
print(s_i)

[
===>el<===
S:: [(e ← ⊤)]
Delta:: [(l | e)]
V:: [(e:None), (l:None)]
, 
===>elo<===
S:: [(e ← ⊤)]
Delta:: [(l | e), (l | o)]
V:: [(e:None), (l:None), (o:None)]
]


<h2> Creating The Goal Condition Gamma</h2>
<p> The goal, gamma, tells us what output from our external activation function (defined next) we want to reproduce. Gamma should almost always correspond to empirical data. In this case, suppression is observed if el believes that she will study late in the library and elo does not. </p>

In [9]:
gamma={'el':'She will study late in the library',
       'elo':'We are uncertain if she will study late in the library'}

<h2> Creating The External Evaluation Function f()</h2>
<p> The external evaluation function f() uses information contained in our SCP to produce output that can be falsified or verified by comparring it to gamma. In our case we will use the final state dependent function <i>f_suppression_studyLate()</i>. Our activation function makes use a second function <i>f_studyLateSingle()</i> which determines, for a single base point in the final epistemic state point, if she will study late in the library.</p>

In [10]:
# Compares the subSCPs 'el' and 'elo' and if there is a response in 'el' that is
# not in 'elo', then we have supressed an inference.
def f_suppression_studyLate(pi):
    finalStructures=pi.evaluate()
    finalStates=StatePointOperations.flattenStatePoint(finalStructures)
    #print (finalStates)
    #get all realised epis with 'el' name 
    statesForCaseEL = StatePointOperations.extractBasePointsFromFlattenedStatePoint(finalStates,name="el")
    #get all realised epis with 'elo' name 
    statesForCaseELO = StatePointOperations.extractBasePointsFromFlattenedStatePoint(finalStates,name="elo")
    
    # find the set of responses that that the realised SCPs of 'el' could reach
    responsesEL = f_studyLateSingle(statesForCaseEL)
    # find the set of responses that that the realised SCPs of 'elo' could reach
    responsesELO = f_studyLateSingle(statesForCaseELO)
    
    #suppression has occured if there is a response in responsesEL which is NOT
    # in responsesELO
    return {'el':responsesEL,'elo':responsesELO}

# suppression is observed when el leads to the inference (l:True) and elo does not
# evaluates a single epistmeic state to model the conclusion
def f_studyLateSingle(finalEpis):
    if not isinstance(finalEpis,list):
        finalEpis=[finalEpis]
    #print("Final states:")
    #print(finalEpis)
    goal2 = [('l',True)]
    responses=[]
    for epi in finalEpis:
        #print ("\n")
        #print (epi)
        V = epi['V']
        
        for var in V:
            for goal in goal2:
                if var.getName()==goal[0] :
                    if var.getValue()==goal[1]:
                        responses.append("She will study late in the library")
                    if var.getValue()!=None and var.getValue()!=goal[1]:
                        responses.append("She will not study late in the library")
                    if var.getValue()==None:
                        responses.append("We are uncertain if she will study late in the library")
        #print ("Epistemic state:")
        #print (epi)
        #print ("response(p_bar):", responses[-1])
        #print ("------------------------------")
    return responses
        

In [11]:
f=f_suppression_studyLate

<h2> The Set of Allowable Cognitive Operation M</h2>
<p>The set of allowable cognitive operation in M determines what actions we can take to model the task at hand. In our case, we will use our suspicion that the task can be modelled using the Weak Completion Semantics. We will add both an aggregate operation for the WCS as a whole, as well as operations for weak completion and the semantic operation (redundant, but interesting).</p>

In [12]:
# THE SET OF COGNITIVE OPERATIONS APPROPRIATE TO THE SUPPRESSION TASK
ADDAB = CognitiveOperation.m_addAB()
WCS = CognitiveOperation.m_wcs()
WC = CognitiveOperation.m_wc()
SEMANTIC = CognitiveOperation.m_semantic()

In [13]:
M=[ADDAB,WCS,SEMANTIC,WC]

 <h2> The SCP Task </h2>
<p>The SCP Task describes the cognitive task that we wish to model in its entirety. An SCP Task consists of:</p>
<ul>
<li><b>initial state point</b>: Our initial information about the world.</li>
<li><b>M</b>: A set of allowable cognitive operations.</li>
<li><b>f()</b>: Our external activation function.</li>
<li><b>Gamma</b>: The empirical result we wish to model.</li>


In [14]:
print (s_i)
print (M)
print (f)
print (gamma)

[
===>el<===
S:: [(e ← ⊤)]
Delta:: [(l | e)]
V:: [(e:None), (l:None)]
, 
===>elo<===
S:: [(e ← ⊤)]
Delta:: [(l | e), (l | o)]
V:: [(e:None), (l:None), (o:None)]
]
[addAB, wcs, semantic, wc]
<function f_suppression_studyLate at 0x000001F414CE34C8>
{'el': 'She will study late in the library', 'elo': 'We are uncertain if she will study late in the library'}


In [15]:
task = SCP_Task.SCP_Task(s_i,M,f,gamma)  

<h2> SCP Search </h2>
<p>In order to find SCPs which model our task, we conduct a <i>De Novo</i> from the initial state until a certain depth. Every SCP produced by appending cognitive operations from M to the CTM which starts with s_i is a candidate solution. The type of validity and optimality requirements we use can limit this list. In our case, we simply wish to conduct a search for any case in which suppression occurs.</p>

In [16]:
searchResult = task.deNoveSearch(depth=3)

In [17]:
for result in searchResult:
    print (result)

(si => addAB => addAB => wcs, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => addAB => semantic, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => wcs => addAB, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => wcs => wcs, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => wcs => semantic, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => wcs => wc, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => semantic => addAB, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => semantic => wcs, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => semantic => semantic, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => semantic => wc, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB => wc => wcs, <function f_suppression_studyLate at 0x000001F414CE34C8>)
(si => addAB =

We have now found several SCPs which, when applied, demonstrate suppression.

<h2> Examining a single SCP</h2>
<p>Let us now focus on one of these candidate solutions: <i>si => addAB => semantic => wc </i>. We will create a <i>CTM</i> which describes this sequence of congitive operation, and we will show what prediction f(CTM) makes and why they meet the goal gamma.</p>

In [18]:
def standardSuppression(s_i):
    f=f_suppression_studyLate
    #The desired output of the external evaluation function
    gamma={'el':'She will study late in the library',
           'elo':'We are uncertain if she will study late in the library'}
    
    #test ctm
    c = CTM.CTM()
    c.setSi(s_i)
    c.appendm(ADDAB)
    c.appendm(WC)
    c.appendm(SEMANTIC)
    
    #final epistemic states of the scp
    final=c.evaluate()
    print ("Final epistemic states are:")
    for state in final:
        print (state)
    predictions = f(c)
    print ("")
    print ('predictions: ', predictions)
    print ('goal (gamma): ', gamma)
    print ("f(pi) models_lenient gamma_Sup :", StatePointOperations.predictionsModelsGamma_lenient(predictions,gamma))
    print ("f(pi) models_strict gamma_Sup : ", 
           (StatePointOperations.predictionsModelsGamma_lenient(predictions,gamma)))  

In [19]:
standardSuppression(s_i)

Final epistemic states are:
[[[
===>el<===
S:: [(e ↔ ⊤), (l ↔ (e ∧ (¬ ab_1))), (ab_1 ↔ ⊥)]
Delta:: []
V:: [(e:True), (l:True), (ab_1:False)]
]]]
[[[
===>elo<===
S:: [(e ↔ ⊤), (l ↔ ((e ∧ (¬ ab_1)) ∨ (o ∧ (¬ ab_2)))), (ab_1 ↔ (¬ o)), (ab_2 ↔ (¬ e))]
Delta:: []
V:: [(e:True), (l:None), (o:None), (ab_1:None), (ab_2:False)]
]]]

predictions:  {'el': ['She will study late in the library'], 'elo': ['We are uncertain if she will study late in the library']}
goal (gamma):  {'el': 'She will study late in the library', 'elo': 'We are uncertain if she will study late in the library'}
f(pi) models_lenient gamma_Sup : True
f(pi) models_strict gamma_Sup :  True


It is clear that, for this SCP, suppression has been observed because l is true in el and l is unknown in elo.